# Project Work - IRM24
## Jürgen Aumayr & Natalia Trudova

## Random Forests

This code demonstrates how to create a random forest classifier using scikit-learn's RandomForestClassifier class.

In [4]:
# Import libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load the iris dataset
iris = load_iris()
X2, y2 = iris.data, iris.target

# Split the data into train and test sets
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.5, random_state=42)

# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf.fit(X2_train, y2_train)

RandomForestClassifier(random_state=42)

## Evaluating Model

Once we have trained our models, it's essential to evaluate their performance on a separate test set. This allows us to assess how well the models generalize to unseen data. Common evaluation metrics include accuracy, precision, recall, and F1-score.

In [5]:
# Import libraries
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluate random forest
y_pred_rf = rf.predict(X2_test)
accuracy_rf = accuracy_score(y2_test, y_pred_rf)
precision_rf = precision_score(y2_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y2_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y2_test, y_pred_rf, average='weighted')


print("Random Forest:")
print("---------------------------------------")
print("Accuracy:  " + str(accuracy_rf))
print("Precision: " + str(precision_rf))
print("Recall:    " + str(recall_rf))
print("F1-Score:  " + str(f1_rf) + "\n\n")


Random Forest:
---------------------------------------
Accuracy:  0.9866666666666667
Precision: 0.9872222222222223
Recall:    0.9866666666666667
F1-Score:  0.9866603624901497


